# Demo: Serenity Derivatives API Function Showcase

Serenity builds in sophisticated option and rates analytics as part of its core offering, and these functions
are all exposed via the API. This notebook shows how to use API functions.

In [ ]:
%%capture --no-stderr --no-display
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from serenity_sdk.widgets import ConnectWidget

# if you want to auto-connect, set this environment variable to your desired default
connect_widget = ConnectWidget(os.getenv('SERENITY_CONFIG_ID', None))

In [ ]:
from datetime import datetime, timedelta
from uuid import UUID, uuid4
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import random

from serenity_types.pricing.derivatives.rates.yield_curve import (
    CurveUsage, 
    YieldCurveVersion, InterpolatedYieldCurve, 
    InterpolationMethod, YieldCurveDefinition)
from serenity_types.pricing.derivatives.options.valuation import (
    DiscountingMethod, 
    MarketDataOverride, 
    YieldCurveOverride,
    OptionValuationRequest, 
    OptionValuation
    )
from serenity_types.pricing.derivatives.options.volsurface import (
    StrikeType, VolatilitySurfaceDefinition,
    InterpolatedVolatilitySurface, VolModel, DiscountingMethod, ProjectionMethod
)

import serenity_sdk.renderers.derivatives.converters as rcvtr

# plot parameters
plt.rcParams['font.size'] = '16'

# create an alias to the api
api = connect_widget.get_api()

# Select underlier asset and option
Select an underlier asset and an option to use in the rest of this notebook.

In [ ]:
# Get all supported underlier assets
api.pricer().get_supported_underliers()

In [ ]:
# Let's pick a underlier to use below. 
underlier_asset_uuid = UUID('78e2e8e2-419d-4515-9b6a-3d5ff1448e89') # BTC uuid

In [ ]:
# get all supported options o f the selected underlier
supported_options = api.pricer().get_supported_options(underlier_asset_id=underlier_asset_uuid)
print('Showing a sample supported option: ')
supported_options[-1]

In [ ]:
# Let's pick an option to use below. 
selected_option = supported_options[-1]
selected_option_uuid = selected_option.asset_id
selected_option

# Available yield curves

In [ ]:
start_dt, end_dt = (datetime.utcnow() - timedelta(hours=5)), datetime.utcnow()

In [ ]:
# as of the latest, without UUID, show all curves
versions = api.pricer().get_available_yield_curve_versions()
print(f"# of versions: {len(versions)}")
print(versions)

#### Let's keep the relevant curves for option valuations later in this notebook
yc_proj_version = [v for v in versions if v.definition.underlier_asset_id==underlier_asset_uuid][0]
yc_disc_version = [v for v in versions if v.definition.curve_usage==CurveUsage.DISCOUNTING][0]
disc_underlier_uuid = yc_disc_version.definition.underlier_asset_id

In [ ]:
# as of the latest, with UUID
data_id = versions[0].definition.yield_curve_id
versions = api.pricer().get_available_yield_curve_versions(yield_curve_id=data_id)
print(f"With UUID = {data_id}, # of versions: {len(versions)}")
versions

In [ ]:
# with the time period
versions = api.pricer().get_available_yield_curve_versions(start_datetime=start_dt, end_datetime=end_dt)
versions = api.pricer().get_available_yield_curve_versions(yield_curve_id = data_id, start_datetime=start_dt, end_datetime=end_dt)

# Available volatlity surfaces

In [ ]:
# as of the latest, without UUID, show all vol surface
versions = api.pricer().get_available_volatility_surface_versions()
print(f"# of versions: {len(versions)}")
print(versions)

#### Let's keep the relevant volatility surface for option valuations later in this notebook
vol_surf_version = [v for v in versions if v.definition.underlier_asset_id==underlier_asset_uuid and v.definition.strike_type==StrikeType.LOG_MONEYNESS][0]

In [ ]:
# as of the latest, with UUID
data_id = versions[0].definition.vol_surface_id
versions = api.pricer().get_available_volatility_surface_versions(vol_surface_id=data_id)
print(f"With UUID = {data_id}, # of versions: {len(versions)}")
versions


In [ ]:
# with the time period
versions = api.pricer().get_available_volatility_surface_versions(start_datetime=start_dt, end_datetime=end_dt)
versions = api.pricer().get_available_volatility_surface_versions(vol_surface_id=data_id, start_datetime=start_dt, end_datetime=end_dt)

# Option valuation

In [ ]:
# selected option valuation using the default option
opt_val = OptionValuation(valuation_id=str(uuid4()),option_asset_id=selected_option_uuid)

## Run different valuation modes
Users can choose to run in 
* Real-time mode: Leave `as_of_time` empty.
* Historical mode: Set `as_of_time`
And, set the different combinatiions of projection & discounting methods

In [ ]:
results = {}
utc_now = datetime.utcnow()
results['Real,P(C),D(S)'] = api.pricer().compute_option_valuations(request=OptionValuationRequest(
    options=[opt_val], 
    projection_method=ProjectionMethod.CURVE,
    discounting_method=DiscountingMethod.SELF_DISCOUNTING))
results['Real,P(C),D(C)'] = api.pricer().compute_option_valuations(request=OptionValuationRequest(
    options=[opt_val], 
    projection_method=ProjectionMethod.CURVE,
    discounting_method=DiscountingMethod.CURVE))
results['Real,P(F),D(S)'] = api.pricer().compute_option_valuations(request=OptionValuationRequest(
    options=[opt_val], 
    projection_method=ProjectionMethod.FUTURES,
    discounting_method=DiscountingMethod.SELF_DISCOUNTING))
results['Real,P(F),D(C)'] = api.pricer().compute_option_valuations(request=OptionValuationRequest(
    options=[opt_val], 
    projection_method=ProjectionMethod.FUTURES,
    discounting_method=DiscountingMethod.CURVE))
results['Hist,P(C),D(S)'] = api.pricer().compute_option_valuations(request=OptionValuationRequest(
    as_of_time=utc_now,
    options=[opt_val], 
    projection_method=ProjectionMethod.CURVE,
    discounting_method=DiscountingMethod.SELF_DISCOUNTING))
results['Hist,P(C),D(C)'] = api.pricer().compute_option_valuations(request=OptionValuationRequest(
    as_of_time=utc_now,
    options=[opt_val], 
    projection_method=ProjectionMethod.CURVE,
    discounting_method=DiscountingMethod.CURVE))

rcvtr.convert_object_dict_to_df({k:v[0] for k,v in results.items()})

## Specify curve and vol surface with UUIDs

Users cann specify curve and volatility UUIDs

In [ ]:
api.pricer().compute_option_valuations(request=OptionValuationRequest(
    options=[opt_val], 
    projection_method=ProjectionMethod.CURVE,
    discounting_method=DiscountingMethod.CURVE,
    projection_curve_override=YieldCurveOverride(yield_curve_id=yc_proj_version.definition.yield_curve_id),
    discounting_curve_override=YieldCurveOverride(yield_curve_id=yc_disc_version.definition.yield_curve_id),
    vol_surface_id=vol_surf_version.definition.vol_surface_id
    ))

## Use an own volatility surface

Users can construct their own volatilty surfaces using SVI parameters

In [ ]:
# Set up vol surface. For demonstration purpose, we set up a flat volatility surface
vol_level = 0.5
calibration_params={tte: {'a': vol_level**2*tte, 'b': 0.0, 'rho': 0.0, 'm':0.0, 's': 0.0}
    for tte in [0.1, 0.5, 1.0]}

my_vs_def = VolatilitySurfaceDefinition(
    vol_surface_id=str(uuid4()), vol_model=VolModel.SVI, 
    strike_type=StrikeType.LOG_MONEYNESS, underlier_asset_id=underlier_asset_uuid,
    display_name='My VS')
my_interp_vs = InterpolatedVolatilitySurface(definition=my_vs_def,calibration_params=calibration_params,
        strikes=[0,1],time_to_expiries=[0,1],vols=[0,0,0,0],input_params={})

api.pricer().compute_option_valuations(request=OptionValuationRequest(options=[opt_val], vol_surface=my_interp_vs))   

## Spot and Volatillity Overrides

Users can bump/replace spot and volatility

In [ ]:
opt_val_with_overrides = OptionValuation(valuation_id=str(uuid4()), option_asset_id=selected_option_uuid, 
    spot_price_override=MarketDataOverride(additive_bump=1000), implied_vol_override=MarketDataOverride(replacement=0.1))

api.pricer().compute_option_valuations(request=OptionValuationRequest(options=[opt_val_with_overrides]))   

## Yield curve overrides
Yield cuve can be overridden through
* creating its own yield curve using `InterpolatedYieldCurve` object.
* applying an additive bump using `MarketDataOverride`
* replacing to a flat curve using `MarketDataOverride` 

In [ ]:
# own curve
yc_def = YieldCurveDefinition(
    yield_curve_id = str(uuid4()), 
    curve_usage=CurveUsage.PROJECTION, 
    interpolation_method="FLAT_FWD", 
    rate_source_type="FUTURE_PX", 
    underlier_asset_id=underlier_asset_uuid,
    display_name="My Proj Curve"
)
durations = [0.1, 1.0]
rates = [0.05, 0.05]
dfs = [np.exp(-d*r) for d,r in zip(durations, rates)]
my_curve = InterpolatedYieldCurve(definition = yc_def, durations=durations, rates=rates, discount_factors=dfs)

In [ ]:
# define yield curve override objects
yc_override_bump = YieldCurveOverride(rate_override=MarketDataOverride(additive_bump=0.02))
yc_override_replace = YieldCurveOverride(rate_override=MarketDataOverride(replacement=0.06))
yc_override_curve = YieldCurveOverride(yield_curve=my_curve)

opt_val_with_overrides = OptionValuation(valuation_id=str(uuid4()), option_asset_id=selected_option_uuid, 
    spot_price_override=MarketDataOverride(additive_bump=1000), implied_vol_override=MarketDataOverride(replacement=0.1)
    )

r = api.pricer().compute_option_valuations(request=OptionValuationRequest(options=[opt_val_with_overrides], 
    projection_method=ProjectionMethod.CURVE, discounting_method=DiscountingMethod.CURVE)
    )
print(f"Without yield curve overrides: {r}")

r = api.pricer().compute_option_valuations(request=OptionValuationRequest(options=[opt_val_with_overrides], 
    projection_method=ProjectionMethod.CURVE, discounting_method=DiscountingMethod.CURVE, 
    projection_curve_override=yc_override_bump, discounting_curve_override=yc_override_replace) 
    )
print(f"Using MarketDataOverrides (bump and replacement): {r}")


r = api.pricer().compute_option_valuations(request=OptionValuationRequest(options=[opt_val_with_overrides], 
    projection_method=ProjectionMethod.CURVE, discounting_method=DiscountingMethod.CURVE, 
    projection_curve_override=yc_override_curve, discounting_curve_override=yc_override_replace) 
    )
print(f"Using own InterpolatedYieldCurve object and MarketDataOverride object (replace): {r}")

# END